# Homework 3
## Kozubek Hubert

### Importing libraries

In [132]:
import pandas as pd
import pickle as pkl
import numpy as np
import dalex as dx
from math import pi
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [133]:
random.seed(123)

### Data preprocessing

In [134]:
df = pd.read_csv('../EPL_2021.csv')

In [135]:
# X = df.copy()

In [136]:
df = df[df['result'] !=  'OwnGoal']
df['result'] = [1 if a == 'Goal' else 0 for a in df['result']]

In [137]:
df = df.assign(distanceToGoal = ((105 - (df['X'] * 105)) ** 2 + (32.5 - (df['Y'] * 68)) ** 2) ** 0.5)
df = df.assign(angleToGoal = np.abs(np.arctan((7.32 * (105 - (df['X'] * 105))) / ((105 - (df['X'] * 105))**2 + (32.5 - (df['Y'] * 68)) ** 2 - (7.32 / 2) ** 2)) * 180 / pi))
df = df[['result', 'h_a', 'situation', 'shotType', 'lastAction', 'minute', 'distanceToGoal', 'angleToGoal']]

In [138]:
label_encoder = LabelEncoder()
categorical_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
changed_from =[]
changed_to = []
for feature in categorical_features:
    print(feature, list(df[feature].unique()), list(label_encoder.fit_transform(df[feature].unique())), "\n")
    changed_from.append(list(df[feature].unique()))
    changed_to.append(list(label_encoder.fit_transform(df[feature].unique())))
for feature in categorical_features:
    df[feature]=label_encoder.fit_transform(df[feature])

h_a ['h', 'a'] [1, 0] 

situation ['OpenPlay', 'FromCorner', 'DirectFreekick', 'SetPiece', 'Penalty'] [2, 1, 0, 4, 3] 

shotType ['Head', 'RightFoot', 'LeftFoot', 'OtherBodyPart'] [0, 3, 1, 2] 

lastAction ['Aerial', 'Throughball', 'BallRecovery', 'Pass', 'Chipped', 'Cross', 'BallTouch', 'Standard', 'None', 'TakeOn', 'HeadPass', 'Rebound', 'LayOff', 'Dispossessed', 'Goal', 'Tackle', 'Foul', 'End', 'CornerAwarded', 'BlockedPass', 'SubstitutionOn', 'KeeperPickup', 'Interception', 'OffsidePass', 'Clearance', 'Card', 'Challenge'] [0, 26, 1, 20, 6, 9, 2, 22, 18, 25, 14, 21, 17, 10, 13, 24, 12, 11, 8, 3, 23, 16, 15, 19, 7, 4, 5] 



In [139]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('result', axis=1),df['result'],test_size=0.2)

### Training Models

In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### Logistic Regression

In [141]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print (f'Logistic Regression score: {lr.score(X_test, y_test)*100:.2f}%')

Logistic Regression score: 90.16%


#### Random Forest

In [142]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print (f'Random Forest score: {rf.score(X_test, y_test)*100:.2f}%')

Random Forest score: 91.04%


#### XGBoost

In [143]:
xgb = GradientBoostingClassifier()
xgb.fit(X_train, y_train)
print (f'XGBoost score: {xgb.score(X_test, y_test)*100:.2f}%')

XGBoost score: 91.38%


### Predictions

In [144]:
obs_id = 420
obs = X_train.iloc[[obs_id]]
# print(f"Logistic Regression probability: {lr.predict_proba(obs)[0][1]*100:.2f}%")
# print(f"Random Forest probability: {rf.predict_proba(obs)[0][1]*100:.2f}%")
print(f"XGBoost probability: {xgb.predict_proba(obs)[0][1]*100:.2f}%")
print(f"Correct label: {y_train.iloc[obs_id]}")


XGBoost probability: 5.34%
Correct label: 0


In [145]:
import dalex as dx
gb_exp = dx.Explainer(xgb, X_train, y_train, label = "Gradient Boosting")

Preparation of a new explainer is initiated

  -> data              : 5888 rows 7 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 5888 values
  -> model_class       : sklearn.ensemble._gb.GradientBoostingClassifier (default)
  -> label             : Gradient Boosting
  -> predict function  : <function yhat_proba_default at 0x0000025DD375B700> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00845, mean = 0.106, max = 0.953
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.797, mean = -8.64e-05, max = 0.982
  -> model_info        : package sklearn

A new explainer has been created!


### CP Profiles

In [146]:
cp_obs_gb = gb_exp.predict_profile(obs)
cp_obs_gb.plot()

Calculating ceteris paribus: 100%|██████████| 7/7 [00:00<00:00, 279.99it/s]


In [147]:
lr_exp = dx.Explainer(lr, X_train, y_train, label = "Logistic Regression")
rf_exp = dx.Explainer(rf, X_train, y_train, label = "Random Forest")

Preparation of a new explainer is initiated

  -> data              : 5888 rows 7 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 5888 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : Logistic Regression
  -> predict function  : <function yhat_proba_default at 0x0000025DD375B700> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000188, mean = 0.105, max = 0.735
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.722, mean = 1.65e-06, max = 0.99
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 5888 rows 7 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Co

In [148]:
cp_obs_lr = lr_exp.predict_profile(obs)
cp_obs_rf = rf_exp.predict_profile(obs)
cp_obs_gb.plot([cp_obs_rf, cp_obs_lr])

Calculating ceteris paribus: 100%|██████████| 7/7 [00:00<00:00, 66.01it/s]


After viewing the results, we can see that Random Forest and XGBoost are mostly similar. On all variables we can see stairs that are characteristic do this models. Logistic Regression on the other hand is pretty smooth. This is for sure the trait of the model.

Probably the main difference in all models is distance to goal variable. Its really intreating how Gradient boosting seem to give better prediction for shots further away. It shows that although with the best score this model may be poor explainer of the real word cause and effect. We need to remember although, that CP profiles don't take into account corelation between variables.

The results show us that even tho model scores are pretty similar, the CP profiles are very different. However we can see how complicated are some of the models and also that random forest witch came in second has much more similar CP profile to the best's model CP, than Logistic regression. I